# Data Preparation for the Nord_H2ub Spine Model

This jupyter notebook contains all routines for the preparation of the input data sources into a input data file for the model in Spine. 

**Authors:** Johannes Giehl (jfg.eco@cbs.dk), Dana Hentschel (djh.eco@cbs.dk)

## General settings

### Packages:

In [2]:
import sys
import os

#the following is necessary to import functions from subfolders
# Get the current directory of the Jupyter Notebook

notebook_dir = os.getcwd()

# Construct the path to the subfolder
subfolder_path = os.path.join(notebook_dir, '00_functions')

# Add the subfolder to the system path
if subfolder_path not in sys.path:
    sys.path.append(subfolder_path)
    
import pandas as pd
from datetime import timedelta
import papermill as pm
from nord_h2ub_dropdown_functions import *

### Base Parameters:

In [3]:
# Example usage
combined_tabs, dropdowns = create_combined_dropdowns_tabs()

In [4]:
#get the values after setting the inputs by the user
values = get_dropdown_values(dropdowns)

#Calculate remaining values based on inputs
calc_values = compute_other_values(values)

### Dictionary:

In [ ]:
base_parameters = {
    'year': values['year'],
    'start_date': calc_values['start_date'],
    'end_date': calc_values['end_date'],
    'area': values['price_zone'],
    'product': values['product'],
    'scenario': [values['base_scen']] + values['other_scen'].split('\n'),
    'frequency': values['frequency'],
    'model_name': values['model_name'],
    'temporal_block': calc_values['temporal_block'],
    'stochastic_scenario': values['stoch_scen'],
    'stochastic_structure': values['stoch_struc'],
    'report_name': values['report_name'],
    'reports': values['outputs'],
    'electrolyzer_type': values['electrolysis'],
    'des_segments_electrolyzer': values['des_segments_electrolyzer'],
    'share_of_dh_price_cap': values['share_of_dh_price_cap'],
    'price_level_power': values['number_price_level_power'],
    'power_price_variance': values['power_price_variance'],
    'roll_forward_use': values['roll_forward'],
    'roll_forward_size': calc_values['roll_forward_size'],
    'num_slices': values['num_slices'],
    'num_steps': calc_values['num_steps']
}

### Execute standard routine:

In [ ]:
pm.execute_notebook(
    'data_preparation_nord_h2ub_spine.ipynb',
    'please_ignore.ipynb',
    parameters=base_parameters
)